In [10]:
import tensorflow as tf  
import sys  
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

In [6]:
mnist = input_data.read_data_sets('data/fashion', one_hot=True)
FLAGS = tf.app.flags.FLAGS
# 初次启动下载相关资料
tf.app.flags.DEFINE_string('CNN', 'cnn/',"""存放模型的目录""")
tf.app.flags.DEFINE_string('cnn', 'mnist',"""模型的名称""")

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


ArgumentError: argument --CNN: conflicting option string: --CNN

In [11]:
def weight_variable(shape):  
    # 使用截断正态分布生成卷积核
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  
  
def bias_variable(shape):  
    # 使用relu激活函数，用一个正偏置值较准
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
    #定义卷积层
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    # 定义池化层
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  

In [12]:
#模型文件所在的文件夹，若不存在则自动创建
ckpt = tf.train.latest_checkpoint(FLAGS.CNN)
if not ckpt:
    if not os.path.exists(FLAGS.CNN):
        os.mkdir(FLAGS.CNN)

In [13]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# 生成
W = tf.Variable(tf.zeros([784,10]))  
b = tf.Variable(tf.zeros([10]))  

# 第一层 先卷积再池化，卷积核尺寸为5*5, 通道数为1，输出通道为32，池化输出后应为14*14*32
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
# 格式转换
x_image = tf.reshape(x, [-1, 28, 28, 1])  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  

# 第二层 先卷积再池化，卷积核尺寸为5*5, 输入通道为32，输出通道为64，池化输出后应为7*7*64
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
# 此时图片格式简化为7*7 进入第三层全连接层,输入维数7*7*64, 输出维数为1024
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
keep_prob = tf.placeholder("float")  
# 使用drop out防止过拟合
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  

# 第四层，输入1024维，输出10维，即0~9分类
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])   
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
  
# 损失函数，交叉熵 
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
# 使用adam优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) 
# 准确度计算
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  

# 数据存储并更新
test_acc_sum = tf.Variable(0.0)
batch_acc = tf.placeholder(tf.float32)
new_test_acc_sum = tf.add(test_acc_sum, batch_acc)
update = tf.assign(test_acc_sum, new_test_acc_sum)
saver=tf.train.Saver(max_to_keep=2)

# 变量初始化，此为训练用代码
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(FLAGS.CNN,sess.graph)
    ckpt = tf.train.latest_checkpoint(FLAGS.CNN)
    step=0
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(FLAGS.CNN, FLAGS.cnn)) + 1:])
    for i in range(20000):  
        batch = mnist.train.next_batch(50)

        if i%100 == 0:  
            train_accuracy = accuracy.eval(feed_dict={  
                x:batch[0], y_: batch[1], keep_prob: 1.0})  
            print("step %d, training accuracy %.3f"%(i, train_accuracy))
            ckptname=os.path.join(FLAGS.CNN, FLAGS.cnn)
            saver.save(sess,ckptname,global_step=i)
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  

FileNotFoundError: [Errno 2] No such file or directory: './images/test_image.bin'